File notebook này có tác dụng tiền xử lý file bao gồm hai công đoạn:




In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df=pd.read_csv('../Datasets/air_quality.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_filtered=df[df['county']=='Taipei City']


In [ ]:
df_filtered.info()

In [ ]:
df_filtered['sitename'].unique()

In [ ]:
df_filtered=df_filtered[df_filtered['sitename']=='Shilin']

In [ ]:
df_filtered.info()

In [ ]:
df_after_removecol = df_filtered.drop(columns=['latitude', 'longitude','siteid','unit','county'])

In [ ]:
data=df_after_removecol.dropna(subset=['pm2.5','co_8hr','no2','o3','so2','pm10','aqi','co','windspeed','winddirec','pm2.5_avg','o3_8hr','status'])
data['pollutant'] = data['pollutant'].fillna('Normal')

In [ ]:

data.head(20)

In [ ]:
data.info()

In [ ]:
data.to_csv('air_quality_shilin.csv',index=False,sep=",")